# Extra Credit
## Project option 1 - Drug annotation of 23andme report


Columns in the **23andme_v5_hg19_ref.txt** file are:
- CHR: the chromosome number
- POS: the position of the variant - position where the difference in nucleotides is found
- dbSNP_ID: the variant identifier in the dbSNP database (https://www.ncbi.nlm.nih.gov/snp/) (this field will contain a dot . when the SNP is not available in the dbSNP database)
- **ALLELE_23andme** : ALLELE: the nucleotides found at that position on the pair of chromosomes


Columns in the **var_drug_ann.tsv** file are:
- Variant Annotation ID: unique ID number for each variant/drug annotation    
- **dbSNP ID** : Variant/Haplotypes: dbSNP ID or haplotype(s)
- **GENE_SYMBOL** : Gene: HGNC symbol
- **DRUG_NAME** : Drug(s): Drug name
- **PMID** : PubMed identifier
- **PHENOTYPE_CATEGOERY**: Phenotype Category: options [efficacy, toxicity, dosage, metabolism/PK other]
- **SIGNIFICANCE**: Significance: yes or no – determined by if the author stated the association was significant
- **NOTES** :Notes: curator notes field
- **SENTENCE**: Sentence: structured sentence
- **ALLELE_PharmGKB**: Alleles: variant alleles in annotation
- Specialty Population: tags for any special populations this annotation is relevant to (e.g. pediatric)

And other columns: Metabolizer types, isPlural, Is/Is Not associated, Direction of effect, PD/PK terms, Multiple drugs And/or, Population types, Population Phenotypes or diseases, Multiple phenotypes or diseases And/or, Comparison Allele(s) or Genotype(s), Comparison Metabolizer types

### Q1. Map/merge the 23andme file and the variant-drug annotation file based on dbSNP_ID (also known as rsID).

The merged result should have the following columns:

dbSNP_ID, GENE_SYMBOL, DRUG_NAME, PMID, PHENOTYPE_CATEGORY, SIGNIFICANCE, NOTES, SENTENCE, ALLELE_PharmGKB (variant alleles in annotation), ALLELE_23andme (variant alleles in 23andme file)

### Q2. Filter the output so that it only contains significant associations (SIGNIFICANCE is yes) for variants that affect the drug efficacy (`PHENOTYPE_CATEGORY` is `efficacy`).

### Q3. Save the output of the filtering step in a tab-separated file (23andme_PharmGKB_map.tsv) with the following columns:
dbSNP_ID, GENE_SYMBOL, DRUG_NAME, NOTES, SENTENCE, ALLELE_PharmGKB, ALLELE_23andme

### Q4. Create a tab separated file (23andme_PharmGKB_summary.tsv) with summarized data with the following columns:
GENE_SYMBOL, DRUG_NAME, dbSNP_IDs (list of IDs separated by ";")

### Q5. Plot the distribution (histogram) of the number of drugs associated with a gene, and the number of SNPs for a gene.